In [22]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
from math import log

from qiskit import Aer, execute

from qiskit_ionq_provider import IonQProvider 

#Call provider and set token value
provider = IonQProvider(token='BFmvdArkiCbsS12r4LZf5VgYDo4HngsS')

In [26]:
from qiskit import *
from qiskit.circuit.library.standard_gates import RYGate
import numpy as np

def bitstring(num, n):
    bstring = '0'*n + bin(num)[2:]
    return bstring[-n:]

def crypto_hash(M, K):
    """
    M: length n bit-string message to encode
    K: list of d integers from 0 to N-1
    """
    n = len(M) # num bits in message
    d = len(K) # num keys
    N = 2**n
    
    
    num_qubits = int(log(d, 2)) + 1 #?????
    qr = QuantumRegister(num_qubits)
    qc = QuantumCircuit(qr)
    
    qc.h(range(num_qubits-1))

    for j in range(len(M)):
        if M[j] == '1':
            for i in range(d):
                ctrl_state = bitstring(i, num_qubits-1)

                theta = 4*np.pi * int(K[i],2) * 2**j / N
                
                Y_Rotate_Gate = RYGate(theta).control(num_qubits-1, ctrl_state=ctrl_state)
                qc.compose(Y_Rotate_Gate, qr, inplace=True)
    
    #print(qc.draw())
    
    backend = Aer.get_backend('statevector_simulator')
    state = execute(qc,backend).result().get_statevector()
    return(state)
    
    

In [33]:
#state = crypto_hash('1001', ['1011', '1101', '1001', '0010'])

# from qiskit_textbook.tools import array_to_latex
# array_to_latex(state, pretext="\\text{Statevector} = ")

       ┌───┐                                                            »
q46_0: ┤ H ├──────o────────────■───────────o───────────■──────────o─────»
       ├───┤      │            │           │           │          │     »
q46_1: ┤ H ├──────o────────────o───────────■───────────■──────────o─────»
       └───┘┌─────┴─────┐┌─────┴─────┐┌────┴─────┐┌────┴────┐┌────┴────┐»
q46_2: ─────┤ RY(11π/4) ├┤ RY(13π/4) ├┤ RY(9π/4) ├┤ RY(π/2) ├┤ RY(22π) ├»
            └───────────┘└───────────┘└──────────┘└─────────┘└─────────┘»
«                                       
«q46_0: ─────■──────────o─────────■─────
«            │          │         │     
«q46_1: ─────o──────────■─────────■─────
«       ┌────┴────┐┌────┴────┐┌───┴────┐
«q46_2: ┤ RY(26π) ├┤ RY(18π) ├┤ RY(4π) ├
«       └─────────┘└─────────┘└────────┘
